In [1]:
import os
import json
import pandas as pd
from glob import glob

In [3]:
# Get a list of all JSON files in the folder
file_paths2023 = glob(os.path.join('data/2023/','*'))
file_paths2022 = glob(os.path.join('data/2022/','*'))
file_paths2021 = glob(os.path.join('data/2021/','*'))
file_paths2020 = glob(os.path.join('data/2020/','*'))
file_paths2019 = glob(os.path.join('data/2019/','*'))
file_paths2018 = glob(os.path.join('data/2018/','*'))

file_paths = file_paths2023 + file_paths2022 + file_paths2021 + file_paths2020 + file_paths2019 + file_paths2018

# Initialize a list to store data from each file
all_data = []

# Loop through each file and load data
for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        all_data.append(data)

In [15]:
# Initialize a list to hold extracted data
subject_data = []

# Loop through each loaded JSON data and extract relevant fields
for data in all_data:
    response = data.get('abstracts-retrieval-response', {})
    subject_areas = response.get('subject-areas', {})
    subject_area = subject_areas.get("subject-area", [])
    
    for area in subject_area:
        subject_data.append({
            'code': area.get('@code', '').replace(',', '').replace('"', '').replace("'", ''),
            'subject_name': area.get('$', '').replace(',', '').replace('"', '').replace("'", '')
        })

# Convert to DataFrame
df_subject = pd.DataFrame(subject_data)
df_subject.head(10)

,code,subject_name
0,1315,Structural Biology
1,1303,Biochemistry
2,1312,Molecular Biology
3,1602,Analytical Chemistry
4,1607,Spectroscopy
5,1605,Organic Chemistry
6,1604,Inorganic Chemistry
7,1104,Aquatic Science
8,2305,Environmental Engineering
9,2304,Environmental Chemistry


In [16]:
df_subject.shape

(50064, 2)

In [17]:
df_subject.describe()

,code,subject_name
count,50064,50064
unique,321,321
top,1000,Multidisciplinary
freq,1088,1088


In [18]:
df_subject = df_subject.drop_duplicates()
df_subject = df_subject.sort_values(by='code', ascending=True)
df_subject.shape

(321, 2)

In [19]:
df_subject = df_subject.reset_index(drop=True)
df_subject.head(10)

,code,subject_name
0,1000,Multidisciplinary
1,1100,Agricultural and Biological Sciences (all)
2,1101,Agricultural and Biological Sciences (miscella...
3,1102,Agronomy and Crop Science
4,1103,Animal Science and Zoology
5,1104,Aquatic Science
6,1105,Ecology Evolution Behavior and Systematics
7,1106,Food Science
8,1107,Forestry
9,1108,Horticulture


In [20]:
df_subject.to_csv('data/subject_areas.csv', index=False)